2 часть дз
Обучение моделей

Сначала нам нужно подключиться к базе данных и извлечь данные для анализа. Это можно сделать с помощью библиотеки psycopg2 и pandas.

это для сохранения датасета в txt

In [163]:
import psycopg2
import csv

def save_table_data_to_txt():
    conn = psycopg2.connect(
        dbname='oirs_dz3', 
        user='kirillshevchenko', 
        password='root', 
        host='localhost'
    )
    cur = conn.cursor()

    # Выполнение запроса SELECT для получения данных из таблицы
    cur.execute('SELECT * FROM vk_posts')
    table_data = cur.fetchall()

    # Закрытие соединения с базой данных
    cur.close()
    conn.close()

    # Запись данных в файл .txt
    with open('table_data.txt', 'w') as file:
        for row in table_data:
            file.write(','.join(str(cell) for cell in row) + '\n')

    print("Данные из таблицы сохранены в файл table_data.txt")

# Вызов функции для сохранения данных из таблицы в файл
save_table_data_to_txt()

def save_table_data_to_csv():
    conn = psycopg2.connect(
        dbname='oirs_dz3',
        user='kirillshevchenko',
        password='root',
        host='localhost'
    )
    cur = conn.cursor()

    # Выполнение запроса SELECT для получения данных из таблицы
    cur.execute('SELECT * FROM vk_posts')
    table_data = cur.fetchall()

    # Заголовки столбцов для CSV-файла
    headers = ['post_id', 'text', 'likes', 'reposts', 'views']

    # Закрытие соединения с базой данных
    cur.close()
    conn.close()

    # Запись данных в файл CSV
    with open('table_data.csv', 'w', encoding='utf-8', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(headers)  # Запись заголовков столбцов
        writer.writerows(table_data)  # Запись данных

    print("Данные из таблицы сохранены в файл table_data.csv")

# Вызов функции для сохранения данных из таблицы в файл CSV
save_table_data_to_csv()


Данные из таблицы сохранены в файл table_data.txt
Данные из таблицы сохранены в файл table_data.csv


In [164]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string
from pymystem3 import Mystem
import psycopg2
import pandas as pd
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

Собираем текст.


выбираем все посты из бд

In [165]:
data = pd.read_csv('/Users/kirillshevchenko/Desktop/8сем/оирс/3 дз/hw3/table_data.csv')

обрабатываем текст

In [166]:
stop_words = ["и", "в", "во", "не", "что", "он", "на", "я", "с", "со", "как", "а", "то", "все", "она", "так", "его", "но", "да", "ты", "к", "у", "же", "вы", "за", "бы", "по", "только", "ее", "мне", "было", "вот", "от", "меня", "еще", "нет", "о", "из", "ему", "теперь", "когда", "даже", "ну", "вдруг", "ли", "если", "уже", "или", "ни", "быть", "был", "него", "до", "вас", "нибудь", "опять", "уж", "вам", "ведь", "там", "потом", "себя", "ничего", "ей", "может", "они", "тут", "где", "есть", "надо", "ней", "для", "мы", "тебя", "их", "чем", "была", "сам", "чтоб", "без", "будто", "чего", "раз", "тоже", "себе", "под", "будет", "ж", "тогда", "кто", "этот", "того", "потому", "этого", "какой", "совсем", "ним", "здесь", "этом", "один", "почти", "мой", "тем", "чтобы", "нее", "сейчас", "были", "куда", "зачем", "всех", "никогда", "можно", "при", "наконец", "два", "об", "другой", "хоть", "после", "над", "больше", "тот", "через", "эти", "нас", "про", "всего", "них", "какая", "много", "разве", "три", "эту", "моя", "впрочем", "хорошо", "свою", "этой", "перед", "иногда", "лучше", "чуть", "том", "нельзя", "такой", "им", "более", "всегда", "конечно", "всю", "между"]

In [167]:
data['text'] = data['text'].fillna('')  # Замена значений NaN пустыми строками
mystem = Mystem()

def preprocess_text(text):
    text = text.lower()
    lemmas = mystem.lemmatize(text)
    filtered_lemmas = [lemma.strip() for lemma in lemmas if lemma.strip() not in stop_words and lemma.strip() not in string.punctuation]
    filtered_text = ' '.join(filtered_lemmas)
    return filtered_text


Прекрасно! Теперь у вас есть функция `preprocess_text`, которая будет очищать и нормализовать ваш текст:

1. `text.lower()` приводит все буквы в тексте к нижнему регистру. Это уменьшает размер словаря и упрощает дальнейший анализ.

2. `word_tokenize(text)` разбивает текст на отдельные слова, так что вы можете обработать каждое слово по отдельности.

3. `[lemmatizer.lemmatize(word) for word in word_tokens if word not in stop_words and word not in string.punctuation]` это генератор списка, который проходит через каждое слово в `word_tokens`, лемматизирует его (т. е. приводит его к базовой форме) и добавляет его в список, если оно не является стоп-словом или знаком пунктуации.

Обратите внимание, что вы используете `WordNetLemmatizer` из библиотеки NLTK. Этот лемматизатор основан на WordNet, большой лексической базе данных английского языка. Он может быть не очень эффективным для русского языка. Вы можете рассмотреть возможность использования других инструментов для лемматизации русского языка, таких как pymorphy2 или mystem от Яндекса.

удаление шума

In [168]:
# Применить функцию preprocess_text к каждому посту
corpus = data['text'].apply(preprocess_text)

In [169]:
# Преобразование текста в численные векторы с использованием TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

In [170]:
import re

def remove_smileys(text):
    # Паттерн для смайлов в виде эмодзи
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # Смайлы эмодзи (Emoticons)
                               u"\U0001F300-\U0001F5FF"  # Символы и пиктограммы
                               u"\U0001F680-\U0001F6FF"  # Смайлы транспорта и картинки
                               u"\U0001F1E0-\U0001F1FF"  # Смайлы флагов стран
                               u"\U00002600-\U000027BF"  # Смайлы погоды
                               u"\U0001F900-\U0001F9FF"  # Смайлы иконки
                               u"\U0001F601-\U0001F64F"  # Смайлы эмодзи (Emoticons)
                               u"\U0001F680-\U0001F6FF"  # Смайлы транспорта и картинки
                               u"\U000024C2-\U0001F251"  # Смайлы иконки
                               "]+", flags=re.UNICODE)
    
    # Удаление смайлов из текста
    cleaned_text = emoji_pattern.sub(r'', text)
    return cleaned_text

# Применение функции remove_smileys к столбцу 'text' в датасете
data['text'] = data['text'].apply(remove_smileys)


In [131]:
print (data)

       post_id                                               text  likes  \
1     37666179   ОФИЦИАЛЬНО — Месси объявил о переходе в «Инте...   5354   
2     37668297   Официально — Алексис Мак Аллистер — игрок «Ли...    179   
3     37668188           Неплохие условия для Лео \n\n#vkfootball    587   
4     37667981                   Что купит? «МЮ»? \n\n#vkfootball   1572   
5     37667827   Почти половина АПЛ сыграет в еврокубках \n\nВ...    854   
...        ...                                                ...    ...   
1896     13759  Подготовь идеальную презентацию и питчинг прое...      5   
1897     13758  Риск — вечный спутник бизнеса! \n \nКакие инве...     10   
1898     13756  Бизнес-инкубатор ВШЭ приглашает на лекцию Кафе...      3   
1899     13754  Бизнес-инкубатор НИУ ВШЭ представил свои прогр...      3   
1900     13752  Что такое питч и как к нему подготовиться? Рас...     10   

      reposts   views  
1        2008  205518  
2          27    5989  
3          37  

In [132]:
# Определение темы поста на основе ключевых слов
def determine_topic(text):
    business_keywords = ["бизнес", "менедж", "управл", "эконом", "маркет", "бренд", "предприн", "франш", "старт", "коммерц", "инвест", "финанс", "продаж", "стратег", "аналитик", "консульт", "инновац", "рынок", "проект", "индустри", "лидер", "партнер", "финансирован", "организац", "планир", "продукт", "сервис", "клиент", "потребител", "бюджет", "рентабел", "эффективн", "конкурент", "реклам", "сетев", "технолог", "интернет", "платформ", "команда", "персонал", "инновацион", "тренд", "потребност", "производств", "качеств", "стратегическ", "решени", "продвижен", "привлечен", "улучшен", "финтех", "цифров", "инвестор", "развит", "предложен", "бюджетирован", "успех", "консалтинг", "венчур", "индустриализац", "интернационализац", "корпоративн", "финансирован", "риски", "акционер", "современ", "миров", "сегмент", "контракт", "продуктивн", "лидерств", "выгода", "сбыт", "услуг", "интерес", "покупател", "капитал", "эксперт", "инициатив", "возможност", "клиентск", "социальн", "брокер", "портфел", "услов", "конкуренция", "nft", "денег", "вызов", "деньг", "миллион"]
    sports_keywords = ["спорт", "физкультура", "футбол", "хоккей", "тренер", "инструктор", "соревнования", "турнир", "атлетика", "бег", "фитнес", "спорт", "физ", "фут", "хокк", "трен", "соревнован", "тур", "атлет", "фит", "плаван", "баскетбол", "волейбол", "теннис", "бокс", "гимнастика", "шахматы", "бейсбол", "гольф", "сноуборд", "лыжи", "биатлон", "катание", "велоспорт", "бодибилдинг", "скейтборд", "кроссфит", "пауэрлифтинг", "рыбалка", "скалолазание", "бокс", "тайский", "керлинг", "триатлон", "серфинг", "парашют", "дайвинг", "шорт-трек", "паркур", "прыжки", "акробатика", "рыцарские", "игры", "экстрим", "эквестрианство", "кайтсерфинг", "бодигард", "фехтование", "стрельба", "водное", "поло", "фристайл", "аэробика", "фигурное", "конькобежный", "спидвей", "санний", "тяжелая", "атлетик", "перетягивание", "каноэ", "рагби", "флорбол", "сквош", "бейджик", "легкая", "атлетика", "прыжки", "высота", "дзюдо", "каратэ", "баскетбол", "гандбол", "гольф", "стрельба", "спортивная", "гимнастика", "синхронное", "win", "спа", "цска", "апл", "рпл", "лео", "мес", "златан", "ибр", "лч", "лиг", "финал", "реал", "масс", "форм", "го", "игрок", "побед", "финале", "признан", "клуб", "трофей", "куб", "требл", "месси", "холанд"]
    music_keywords = ["исполнител", "альбом", "концерт", "стил", "звучан", "текст", "музыкант", "слушател", "песн", "трек", "испол", "аль", "кон", "стил", "звучан", "текст", "муз", "слуш", "песн", "жанр", "мелод", "артист", "ритм", "аккорд", "дуэт", "сингл", "поп", "рок", "хип-хоп", "электроник", "джаз", "классик", "инструмент", "саксофон", "гитар", "пианин", "ударн", "бас", "вокал", "сол", "сампл", "бит", "микс", "саундтрек", "ремикс", "авторск", "пластинк", "композитор", "студ", "музыкальн", "индустри", "аранжировк", "рефрен", "бэк-вокал", "студийн", "музыкальн", "гармон", "сцен", "музыкальн", "продюсер", "клавишн", "синтезатор", "дискограф", "тур", "кавер", "рок-н-ролл", "поп-рок", "альтернатив", "рэп", "трэп", "регг", "ска", "реггетон", "кантр", "фолк", "инди", "танцевальн", "музык", "клубн", "панк", "метал", "блюз", "эмо", "фанк", "соул", "диско", "рэйв", "психодели", "хаус", "транс", "техн", "драм-н-бэйс", "дабстеп", "джунгл", "реггиджангл", "эмбиент", "ритм-энд-блюз", "гранж", "дэт-метал", "симфон", "оркестр", "опер", "концертн", "зал", "компиляц"]
    politics_keywords = ["государств", "власт", "правительств", "парти", "идеолог", "законодательств", "дипломат", "свобод", "коррупц", "выбор", "кризис", "госу", "влас", "прав", "парти", "идеол", "закон", "дипло", "своб", "корр", "выбор", "кризис", "поб", "сво", "укр", "рос", "вой", "вс", "през", "демократ", "республик", "политик", "политическ", "политичес", "государствен", "правительствен", "дипломатическ", "дипломатичес", "парламент", "правов", "коррупц", "парламентар", "выборн", "конституц", "свободн", "демократическ", "политолог", "политологическ", "парламентск", "политическ", "реформ", "движен", "гражданск", "международн", "конфликт", "договор", "представител", "голосован", "оппозиц", "партийн", "правозащитн", "политсистем", "судебн", "правосуд", "законодател", "декларац", "решен", "международн", "сотрудничеств", "лоббирован", "политическ", "демократизац", "диктатур", "войск", "воен", "граждан", "конституцион", "региональн", "территориальн", "внутренн", "внешн", "геополитическ", "независим", "самоуправл", "безопасн", "суверен", "регулирован", "поддержк", "политическ", "экономическ", "социальн", "росс", "сша", "амер"]
    
    text = text.lower()
    words = word_tokenize(text)

    for word in words:
        for keyword in business_keywords:
            if keyword in word:
                return "Бизнес"
        for keyword in sports_keywords:
            if keyword in word:
                return "Спорт"
        for keyword in music_keywords:
            if keyword in word:
                return "Музыка"
        for keyword in politics_keywords:
            if keyword in word:
                return "Политика"
    return "Другое"

In [106]:
def determine_topic(text):
    business_keywords = ["бизнес", "менедж", "управл", "эконом", "маркет", "бренд", "предприн", "франш", "старт", "коммерц", "инвест", "финанс", "продаж", "стратег", "аналитик", "консульт", "инновац", "рынок", "проект", "индустри", "лидер", "партнер", "финансирован", "организац", "планир", "продукт", "сервис", "клиент", "потребител", "бюджет", "рентабел", "эффективн", "конкурент", "реклам", "сетев", "технолог", "интернет", "платформ", "команда", "персонал", "инновацион", "тренд", "потребност", "производств", "качеств", "стратегическ", "решени", "продвижен", "привлечен", "улучшен", "финтех", "цифров", "инвестор", "развит", "предложен", "бюджетирован", "успех", "консалтинг", "венчур", "индустриализац", "интернационализац", "корпоративн", "финансирован", "риски", "акционер", "современ", "миров", "сегмент", "контракт", "продуктивн", "лидерств", "выгода", "сбыт", "услуг", "интерес", "покупател", "капитал", "эксперт", "инициатив", "возможност", "клиентск", "социальн", "брокер", "портфел", "услов", "конкуренция", "nft", "денег", "вызов", "деньг", "миллион"]
    sports_keywords = ["спорт", "физкультура", "футбол", "хоккей", "тренер", "инструктор", "соревнования", "турнир", "атлетика", "бег", "фитнес", "спорт", "физ", "фут", "хокк", "трен", "соревнован", "тур", "атлет", "фит", "плаван", "баскетбол", "волейбол", "теннис", "бокс", "гимнастика", "шахматы", "бейсбол", "гольф", "сноуборд", "лыжи", "биатлон", "катание", "велоспорт", "бодибилдинг", "скейтборд", "кроссфит", "пауэрлифтинг", "рыбалка", "скалолазание", "бокс", "тайский", "керлинг", "триатлон", "серфинг", "парашют", "дайвинг", "шорт-трек", "паркур", "прыжки", "акробатика", "рыцарские", "игры", "экстрим", "эквестрианство", "кайтсерфинг", "бодигард", "фехтование", "стрельба", "водное", "поло", "фристайл", "аэробика", "фигурное", "конькобежный", "спидвей", "санний", "тяжелая", "атлетик", "перетягивание", "каноэ", "рагби", "флорбол", "сквош", "бейджик", "легкая", "атлетика", "прыжки", "высота", "дзюдо", "каратэ", "баскетбол", "гандбол", "гольф", "стрельба", "спортивная", "гимнастика", "синхронное", "win", "спа", "цска", "апл", "рпл", "лео", "мес", "златан", "ибр", "лч", "лиг", "финал", "реал", "масс", "форм", "го", "игрок", "побед", "финал", "признан", "клуб", "трофей", "куб", "требл", "месси"]
    music_keywords = ["исполнител", "альбом", "концерт", "стил", "звучан", "текст", "музыкант", "слушател", "песн", "трек", "испол", "аль", "кон", "стил", "звучан", "текст", "муз", "слуш", "песн", "жанр", "мелод", "артист", "ритм", "аккорд", "дуэт", "сингл", "поп", "рок", "хип-хоп", "электроник", "джаз", "классик", "инструмент", "саксофон", "гитар", "пианин", "ударн", "бас", "вокал", "сол", "сампл", "бит", "микс", "саундтрек", "ремикс", "авторск", "пластинк", "композитор", "студ", "музыкальн", "индустри", "аранжировк", "рефрен", "бэк-вокал", "студийн", "музыкальн", "гармон", "сцен", "музыкальн", "продюсер", "клавишн", "синтезатор", "дискограф", "тур", "кавер", "рок-н-ролл", "поп-рок", "альтернатив", "рэп", "трэп", "регг", "ска", "реггетон", "кантр", "фолк", "инди", "танцевальн", "музык", "клубн", "панк", "метал", "блюз", "эмо", "фанк", "соул", "диско", "рэйв", "психодели", "хаус", "транс", "техн", "драм-н-бэйс", "дабстеп", "джунгл", "реггиджангл", "эмбиент", "ритм-энд-блюз", "гранж", "дэт-метал", "симфон", "оркестр", "опер", "концертн", "зал", "компиляц"]
    politics_keywords = ["государств", "власт", "правительств", "парти", "идеолог", "законодательств", "дипломат", "свобод", "коррупц", "выбор", "кризис", "госу", "влас", "прав", "парти", "идеол", "закон", "дипло", "своб", "корр", "выбор", "кризис", "поб", "сво", "укр", "рос", "вой", "вс", "през", "демократ", "республик", "политик", "политическ", "политичес", "государствен", "правительствен", "дипломатическ", "дипломатичес", "парламент", "правов", "коррупц", "парламентар", "выборн", "конституц", "свободн", "демократическ", "политолог", "политологическ", "парламентск", "политическ", "реформ", "движен", "гражданск", "международн", "конфликт", "договор", "представител", "голосован", "оппозиц", "партийн", "правозащитн", "политсистем", "судебн", "правосуд", "законодател", "декларац", "решен", "международн", "сотрудничеств", "лоббирован", "политическ", "демократизац", "диктатур", "войск", "воен", "граждан", "конституцион", "региональн", "территориальн", "внутренн", "внешн", "геополитическ", "независим", "самоуправл", "безопасн", "суверен", "регулирован", "поддержк", "политическ", "экономическ", "социальн", "росс", "сша", "амер"]

    text = text.lower()
    words = word_tokenize(text)

    category_count = {
        "Бизнес": 0,
        "Спорт": 0,
        "Музыка": 0,
        "Политика": 0,
        "Другое": 0
    }

    for word in words:
        if word in business_keywords:
            category_count["Бизнес"] += 1
        if word in sports_keywords:
            category_count["Спорт"] += 1
        if word in music_keywords:
            category_count["Музыка"] += 1
        if word in politics_keywords:
            category_count["Политика"] += 1
    if max(category_count.values()) == 0:
        return "Другое"

    max_count = max(category_count.values())
    max_topics = [topic for topic, count in category_count.items() if count == max_count]

    return max_topics[0]  # Возвращаем первую тему с наибольшим количеством совпадений


In [171]:
def determine_topic(text):
    business_keywords = ["бизнес", "менедж", "управл", "эконом", "маркет", "бренд", "предприн", "франш", "старт", "коммерц", "инвест", "финанс", "продаж", "стратег", "аналитик", "консульт", "инновац", "рынок", "проект", "индустри", "лидер", "партнер", "финансирован", "организац", "планир", "продукт", "сервис", "клиент", "потребител", "бюджет", "рентабел", "эффективн", "конкурент", "реклам", "сетев", "технолог", "интернет", "платформ", "команда", "персонал", "инновацион", "тренд", "потребност", "производств", "качеств", "стратегическ", "решени", "продвижен", "привлечен", "улучшен", "финтех", "цифров", "инвестор", "развит", "предложен", "бюджетирован", "успех", "консалтинг", "венчур", "индустриализац", "интернационализац", "корпоративн", "финансирован", "риски", "акционер", "современ", "миров", "сегмент", "контракт", "продуктивн", "лидерств", "выгода", "сбыт", "услуг", "интерес", "покупател", "капитал", "эксперт", "инициатив", "возможност", "клиентск", "социальн", "брокер", "портфел", "услов", "конкуренция", "nft", "денег", "вызов", "деньг", "миллион"]
    sports_keywords = ["спорт", "физкультура", "футбол", "хоккей", "тренер", "инструктор", "соревнования", "турнир", "атлетика", "бег", "фитнес", "спорт", "физ", "фут", "хокк", "трен", "соревнован", "тур", "атлет", "фит", "плаван", "баскетбол", "волейбол", "теннис", "бокс", "гимнастика", "шахматы", "бейсбол", "гольф", "сноуборд", "лыжи", "биатлон", "катание", "велоспорт", "бодибилдинг", "скейтборд", "кроссфит", "пауэрлифтинг", "рыбалка", "скалолазание", "бокс", "тайский", "керлинг", "триатлон", "серфинг", "парашют", "дайвинг", "шорт-трек", "паркур", "прыжки", "акробатика", "рыцарские", "игры", "экстрим", "эквестрианство", "кайтсерфинг", "бодигард", "фехтование", "стрельба", "водное", "поло", "фристайл", "аэробика", "фигурное", "конькобежный", "спидвей", "санний", "тяжелая", "атлетик", "перетягивание", "каноэ", "рагби", "флорбол", "сквош", "бейджик", "легкая", "атлетика", "прыжки", "высота", "дзюдо", "каратэ", "баскетбол", "гандбол", "гольф", "стрельба", "спортивная", "гимнастика", "синхронное", "win", "спа", "цска", "апл", "рпл", "лео", "мес", "златан", "ибр", "лч", "лиг", "финал", "реал", "масс", "форм", "го", "игрок", "побед", "финал", "признан", "клуб", "трофей", "куб", "требл", "месси"]
    music_keywords = ["исполнител", "альбом", "концерт", "стил", "звучан", "текст", "музыкант", "слушател", "песн", "трек", "испол", "аль", "кон", "стил", "звучан", "текст", "муз", "слуш", "песн", "жанр", "мелод", "артист", "ритм", "аккорд", "дуэт", "сингл", "поп", "рок", "хип-хоп", "электроник", "джаз", "классик", "инструмент", "саксофон", "гитар", "пианин", "ударн", "бас", "вокал", "сол", "сампл", "бит", "микс", "саундтрек", "ремикс", "авторск", "пластинк", "композитор", "студ", "музыкальн", "индустри", "аранжировк", "рефрен", "бэк-вокал", "студийн", "музыкальн", "гармон", "сцен", "музыкальн", "продюсер", "клавишн", "синтезатор", "дискограф", "тур", "кавер", "рок-н-ролл", "поп-рок", "альтернатив", "рэп", "трэп", "регг", "ска", "реггетон", "кантр", "фолк", "инди", "танцевальн", "музык", "клубн", "панк", "метал", "блюз", "эмо", "фанк", "соул", "диско", "рэйв", "психодели", "хаус", "транс", "техн", "драм-н-бэйс", "дабстеп", "джунгл", "реггиджангл", "эмбиент", "ритм-энд-блюз", "гранж", "дэт-метал", "симфон", "оркестр", "опер", "концертн", "зал", "компиляц"]
    politics_keywords = ["государств", "власт", "правительств", "парти", "идеолог", "законодательств", "дипломат", "свобод", "коррупц", "выбор", "кризис", "госу", "влас", "прав", "парти", "идеол", "закон", "дипло", "своб", "корр", "выбор", "кризис", "поб", "сво", "укр", "рос", "вой", "вс", "през", "демократ", "республик", "политик", "политическ", "политичес", "государствен", "правительствен", "дипломатическ", "дипломатичес", "парламент", "правов", "коррупц", "парламентар", "выборн", "конституц", "свободн", "демократическ", "политолог", "политологическ", "парламентск", "политическ", "реформ", "движен", "гражданск", "международн", "конфликт", "договор", "представител", "голосован", "оппозиц", "партийн", "правозащитн", "политсистем", "судебн", "правосуд", "законодател", "декларац", "решен", "международн", "сотрудничеств", "лоббирован", "политическ", "демократизац", "диктатур", "войск", "воен", "граждан", "конституцион", "региональн", "территориальн", "внутренн", "внешн", "геополитическ", "независим", "самоуправл", "безопасн", "суверен", "регулирован", "поддержк", "политическ", "экономическ", "социальн", "росс", "сша", "амер"]

    text = text.lower()
    words = word_tokenize(text)

    # Проверяем наличие ключевых слов в тексте
    for word in words:
        for keyword in business_keywords:
            if keyword in word:
                return "Бизнес"
        for keyword in sports_keywords:
            if keyword in word:
                return "Спорт"
        for keyword in music_keywords:
            if keyword in word:
                return "Музыка"
        for keyword in politics_keywords:
            if keyword in word:
                return "Политика"
    
    # Если не было совпадений, используем подсчет количества совпадений
    category_count = {
        "Бизнес": 0,
        "Спорт": 0,
        "Музыка": 0,
        "Политика": 0,
        "Другое": 0
    }

    for word in words:
        if word in business_keywords:
            category_count["Бизнес"] += 1
        if word in sports_keywords:
            category_count["Спорт"] += 1
        if word in music_keywords:
            category_count["Музыка"] += 1
        if word in politics_keywords:
            category_count["Политика"] += 1
    
    max_count = max(category_count.values())
    
    if max_count > 0:
        max_topics = [topic for topic, count in category_count.items() if count == max_count]
        return max_topics[0]  # Возвращаем первую тему с наибольшим количеством совпадений
    
    return "Другое"  # Если нет совпадений, возвращаем "Другое"


In [172]:
# Применить функцию determine_topic к каждому посту
data['topic'] = data['text'].apply(determine_topic)

In [173]:
# Вывести результаты с определенными темами
print(data[['text', 'topic']])

                                                   text   topic
1      ОФИЦИАЛЬНО — Месси объявил о переходе в «Инте...  Музыка
2      Официально — Алексис Мак Аллистер — игрок «Ли...  Музыка
3              Неплохие условия для Лео \n\n#vkfootball  Бизнес
4                      Что купит? «МЮ»? \n\n#vkfootball  Другое
5      Почти половина АПЛ сыграет в еврокубках \n\nВ...   Спорт
...                                                 ...     ...
1896  Подготовь идеальную презентацию и питчинг прое...   Спорт
1897  Риск — вечный спутник бизнеса! \n \nКакие инве...  Бизнес
1898  Бизнес-инкубатор ВШЭ приглашает на лекцию Кафе...  Бизнес
1899  Бизнес-инкубатор НИУ ВШЭ представил свои прогр...  Бизнес
1900  Что такое питч и как к нему подготовиться? Рас...   Спорт

[1900 rows x 2 columns]


In [174]:
def save_table_data_to_csv(data):
    # Выбор только указанных столбцов
    filtered_data = data[['text', 'topic']]

    # Сохранение данных в новый файл CSV
    filtered_data.to_csv('filtered_data.csv', index=False)

    print("Данные сохранены в файл filtered_data.csv")

# Вызов функции для сохранения данных из объекта DataFrame в файл CSV
save_table_data_to_csv(data)

Данные сохранены в файл filtered_data.csv


В данном коде используется модель LSTM (Long Short-Term Memory) для классификации текстовых данных.

Этапы работы с моделью в данном коде:

1. Объединение отсутствующих категорий: Отсутствующие категории в тренировочном и тестовом наборах данных объединяются в категорию "Другое" путем замены соответствующих значений в массивах `X_train`, `y_train`, `X_test` и `y_test`.

2. Кодирование меток классов: Метки классов `y_train` и `y_test` кодируются с помощью объекта `LabelEncoder` для преобразования текстовых меток в числовые значения.

3. Преобразование текста в числовые последовательности: Текстовые данные из `X_train` и `X_test` преобразуются в числовые последовательности с помощью объекта `Tokenizer`. Каждое слово заменяется на соответствующий индекс, полученный из словаря слов `tokenizer.word_index`.

4. Приведение последовательностей к фиксированной длине: Последовательности числовых значений приводятся к фиксированной длине `max_sequence_length` с помощью функции `pad_sequences`. Если последовательность короче, она дополняется нулями, а если длиннее, то обрезается.

5. Создание модели LSTM: Создается объект модели `Sequential`, к которому последовательно добавляются слои. В данном случае добавляются слои `Embedding`, `LSTM` и `Dense`. Слой `Embedding` преобразует индексы слов в векторные представления, слой `LSTM` выполняет обработку последовательностей, а слой `Dense` представляет выходной слой с функцией активации `softmax` для классификации.

6. Компиляция модели: Модель компилируется с функцией потерь `sparse_categorical_crossentropy` и оптимизатором `adam`.

7. Обучение модели: Модель обучается на тренировочных данных `X_train_padded` и `y_train_encoded` с заданным количеством эпох и размером пакета. Валидационные данные `X_test_padded` и `y_test_encoded` используются для проверки модели на каждой эпохе.

8. Оценка модели: Модель оценивается на тестовых данных с использованием функции `evaluate()`, и выводится точность (`accuracy`) модели.

In [176]:
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
import tensorflow as tf

# Разделение датасета на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['topic'], test_size=0.2, random_state=42)

all_categories = ['Бизнес', 'Спорт', 'Музыка', 'Политика']

# Объединение всех отсутствующих категорий в "Другое"
missing_categories_train = set(all_categories) - set(y_train.unique())
if missing_categories_train:
    y_train = y_train.apply(lambda x: 'Другое' if x in missing_categories_train else x)

missing_categories_test = set(all_categories) - set(y_test.unique())
if missing_categories_test:
    y_test = y_test.apply(lambda x: 'Другое' if x in missing_categories_test else x)

# Кодирование меток классов
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Преобразование текста в числовые последовательности
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# Приведение последовательностей к фиксированной длине
max_sequence_length = 100  # Максимальная длина последовательности
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length)

# Создание модели LSTM
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_sequence_length))
model.add(LSTM(units=128))
model.add(Dense(units=len(label_encoder.classes_), activation='softmax'))

# Компиляция модели
#model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#замена
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.legacy.Adam(), metrics=['accuracy'])


# Обучение модели
model.fit(X_train_padded, y_train_encoded, epochs=10, batch_size=32, validation_data=(X_test_padded, y_test_encoded))

# Оценка модели
_, accuracy = model.evaluate(X_test_padded, y_test_encoded)
print('Accuracy:', accuracy)


Epoch 1/10
48/48 [==============================] - 4s 75ms/step - loss: 1.4818 - accuracy: 0.3632 - val_loss: 1.3776 - val_accuracy: 0.4263
Epoch 2/10
48/48 [==============================] - 3s 70ms/step - loss: 1.2131 - accuracy: 0.4717 - val_loss: 1.3007 - val_accuracy: 0.4395
Epoch 3/10
48/48 [==============================] - 4s 78ms/step - loss: 0.7343 - accuracy: 0.7664 - val_loss: 1.3710 - val_accuracy: 0.4605
Epoch 4/10
48/48 [==============================] - 4s 76ms/step - loss: 0.2618 - accuracy: 0.9243 - val_loss: 1.5007 - val_accuracy: 0.4553
Epoch 5/10
48/48 [==============================] - 4s 75ms/step - loss: 0.0719 - accuracy: 0.9868 - val_loss: 1.7711 - val_accuracy: 0.4684
Epoch 6/10
48/48 [==============================] - 4s 74ms/step - loss: 0.0289 - accuracy: 0.9954 - val_loss: 1.9845 - val_accuracy: 0.4816
Epoch 7/10
48/48 [==============================] - 4s 74ms/step - loss: 0.0156 - accuracy: 0.9987 - val_loss: 2.3330 - val_accuracy: 0.4526
Epoch 8/10
48

тестирование 

In [177]:
#это тест на том же самом 
import numpy as np
# Загрузка новых данных для тестирования
new_data = pd.read_csv('/Users/kirillshevchenko/Desktop/8сем/оирс/3 дз/hw3/table_data.csv')

new_data['text'] = new_data['text'].fillna('')

# Предобработка новых данных
new_X = new_data['text']
new_X_sequences = tokenizer.texts_to_sequences(new_X)
new_X_padded = pad_sequences(new_X_sequences, maxlen=max_sequence_length)

# Прогнозирование классов для новых данных
predictions = model.predict(new_X_padded)
predicted_labels = label_encoder.inverse_transform(np.argmax(predictions, axis=1))

# Вывод предсказанных меток классов
print(predicted_labels)

60/60 [==============================] - 1s 21ms/step
['Музыка' 'Музыка' 'Бизнес' ... 'Бизнес' 'Бизнес' 'Спорт']


In [178]:
# Оценка модели на тестовом наборе
_, test_accuracy = model.evaluate(X_test_padded, y_test_encoded)
print('Test Accuracy:', test_accuracy)

# Прогнозирование классов для тестового набора
test_predictions = model.predict(X_test_padded)
test_predicted_labels = label_encoder.inverse_transform(np.argmax(test_predictions, axis=1))

# Вывод предсказанных меток классов на тестовом наборе
print('Test Predictions:', test_predicted_labels)


12/12 [==============================] - 0s 20ms/step - loss: 2.5289 - accuracy: 0.4658
Test Accuracy: 0.46578946709632874
12/12 [==============================] - 0s 24ms/step
Test Predictions: ['Бизнес' 'Политика' 'Политика' 'Бизнес' 'Спорт' 'Спорт' 'Другое' 'Музыка'
 'Музыка' 'Политика' 'Бизнес' 'Музыка' 'Музыка' 'Музыка' 'Спорт' 'Спорт'
 'Спорт' 'Музыка' 'Другое' 'Спорт' 'Другое' 'Спорт' 'Политика' 'Музыка'
 'Политика' 'Другое' 'Политика' 'Политика' 'Другое' 'Спорт' 'Политика'
 'Бизнес' 'Политика' 'Спорт' 'Бизнес' 'Спорт' 'Музыка' 'Политика'
 'Политика' 'Другое' 'Спорт' 'Политика' 'Музыка' 'Музыка' 'Музыка'
 'Музыка' 'Спорт' 'Спорт' 'Спорт' 'Политика' 'Музыка' 'Музыка' 'Спорт'
 'Музыка' 'Политика' 'Музыка' 'Другое' 'Спорт' 'Политика' 'Спорт' 'Спорт'
 'Спорт' 'Музыка' 'Политика' 'Политика' 'Политика' 'Бизнес' 'Музыка'
 'Спорт' 'Бизнес' 'Политика' 'Бизнес' 'Другое' 'Спорт' 'Спорт' 'Спорт'
 'Спорт' 'Спорт' 'Спорт' 'Спорт' 'Политика' 'Музыка' 'Другое' 'Политика'
 'Спорт' 'Политика' 'Му

In [179]:
model.save('path_to_my_model', save_format='tf')

INFO:tensorflow:Assets written to: path_to_my_model/assets


INFO:tensorflow:Assets written to: path_to_my_model/assets


In [180]:
import pickle

# Сохранение токенизатора
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [181]:
# Сохранение LabelEncoder
with open('label_encoder.pickle', 'wb') as le_file:
    pickle.dump(label_encoder, le_file)

